<a href="https://colab.research.google.com/github/Chuyini/Red-Neuronal-LSTM/blob/main/ProjectVersion2LaraVazquez_Jesus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
np.random.seed(4)
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

TypeError: ignored

In [ ]:
# Helper Functions
def plot_predictions(real, prediction):
    plt.plot(real[0:len(prediction)], color='red', label='Real Stock Value')
    plt.plot(prediction, color='blue', label='Stock Value Prediction')
    plt.ylim(1.1 * np.min(prediction) / 2, 1.1 * np.max(prediction))
    plt.xlabel('Time')
    plt.ylabel('Value of Stock')
    plt.legend()
    plt.show()


In [ ]:
# Upload data from MY computer in Python
from google.colab import files
NonLinearData = files.upload()

In [ ]:
# Read data
dataset = pd.read_csv('AAPL_2006-01-01_to_2018-01-01.csv', index_col='Date', parse_dates=['Date'])
dataset.head()

In [ ]:
# Training and validation sets
# LSTM will be trained with data from 2016 and earlier. Validation will be done with data from 2017 and onwards.
# In both cases, only the highest stock value for each day will be used.
training_set = dataset[:'2016'].iloc[:, 1:2]
validation_set = dataset['2017':].iloc[:, 1:2]

In [ ]:
# Normalize the training set
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_training_set = scaler.fit_transform(training_set)

In [ ]:
# The LSTM network will take "time_step" consecutive data points as input and predict 1 data point as output.
# This is how the training set is shaped.
time_step = 60
X_train = []
Y_train = []
m = len(scaled_training_set)

for i in range(time_step, m):
    # X: Blocks of "time_step" data points: 0-time_step, 1-time_step+1, 2-time_step+2, etc.
    X_train.append(scaled_training_set[i - time_step:i, 0])

    # Y: The next data point
    Y_train.append(scaled_training_set[i, 0])
X_train, Y_train = np.array(X_train), np.array(Y_train)

# Reshape X_train to fit the Keras model
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
# LSTM Network
input_dimension = (X_train.shape[1], 1)
output_dimension = 1
units = 65

model = Sequential()
model.add(LSTM(units=units, input_shape=input_dimension))
model.add(Dense(units=output_dimension))
model.compile(optimizer='rmsprop', loss='mse')
model.fit(X_train, Y_train, epochs=20, batch_size=32)

In [ ]:
import theano
theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
theano.config.on_opt_error = 'ignore'

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


units_values = [64, 80, 100, 128]
results = []

for units in units_values:
    model = Sequential()
    model.add(LSTM(units=units, input_shape=input_dimension))
    model.add(Dense(units=output_dimension))
    model.compile(optimizer='rmsprop', loss='mse')
    model.fit(X_train, Y_train, epochs=20, batch_size=32, verbose=0)


    loss = model.evaluate(X_train, Y_train)
    results.append({'Units': units, 'Loss': loss})


results_df = pd.DataFrame(results)
print(results_df)


In [ ]:
!pip install theano==1.0.5

In [ ]:
!pip install numpy==1.19.5

In [ ]:
!pip install numpy==1.21.3

In [ ]:
import os
os.environ["THEANO_FLAGS"] = "blas.ldflags="


In [ ]:
import os
import theano

theano_flags = (
    'floatX=float32,'
    'device=cpu,'
    'force_device=True'
)

os.environ["THEANO_FLAGS"] = theano_flags


In [ ]:
import numpy as np
import pandas as pd
from neupy import algorithms
from sklearn.metrics import mean_squared_error

# Tu código previo para preparar los datos

# Valores de 'units' a probar
units_values = [64, 80, 100, 128]  # Puedes ajustar estos valores según lo que desees probar
results_grnn = []

for units in units_values:
    grnn = algorithms.GRNN(std=0.1, verbose=False)
    grnn.train(X_train, Y_train)
    y_pred_grnn = grnn.predict(X_train)
    loss_grnn = mean_squared_error(Y_train, y_pred_grnn)
    results_grnn.append({'Units': units, 'Loss': loss_grnn})

# Mostrar los resultados en forma de tabla
results_grnn_df = pd.DataFrame(results_grnn)
print(results_grnn_df)

/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


AttributeError: ignored

In [ ]:
# Validation (stock value prediction)
x_test = validation_set.values
x_test = scaler.transform(x_test)

X_test = []
for i in range(time_step, len(x_test)):
    X_test.append(x_test[i - time_step:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

prediction = model.predict(X_test)
prediction = scaler.inverse_transform(prediction)

# Plot results
plot_predictions(validation_set.values, prediction)